# Data Prep+Aug

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import cv2

In [2]:
driving_log = pd.DataFrame.from_csv('sample_data/driving_log.csv')
driving_log.head()

,left,right,steering,throttle,brake,speed
center,,,,,,
IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [3]:
# initial loop, no augmentation

"""
for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    images.append(img)
    measurements.append(float(line[3]))    
"""
print()

In [4]:
# second loopo, added flipping

"""
for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    
    images.append(img)
    measurements.append(float(line[3]))    
        
    images.append(cv2.flip(img,1))
    measurements.append( -1.0 * float(line[3])) 
"""
print()

In [5]:
# third loop, added multiple cameras

CORRECTION = 0.2

images = []
measurements = []

with open('sample_data/driving_log.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader, None)
    #c = 0
    for row in reader:
        img_center = cv2.imread('sample_data/' + row[0].strip())
        img_left = cv2.imread('sample_data/' + row[1].strip())
        img_right = cv2.imread('sample_data/' + row[2].strip())
        img = [img_center, img_left, img_right]
        
        steer_center = float(row[3])
        steer_left = steer_center + CORRECTION
        steer_right = steer_center - CORRECTION
        steer = [steer_center, steer_left, steer_right]

        # normal
        images.extend(img)
        # images.append(img_center)
        measurements.extend(steer)    

        # flipped
        images.extend([cv2.flip(i,1) for i in img])
        measurements.extend([-1.0 * s for s in steer])   
        #c+=1
        #if c > 5000: break

In [6]:
X_train = np.array(images)
y_train = np.array(measurements)

In [7]:
X_train.shape

(48216, 160, 320, 3)

---
# Model Architecture

In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers import Cropping2D

#from keras.layers.pooling

Using TensorFlow backend.


In [9]:
EPOCHS = 10

In [10]:
model = Sequential()

# normalize
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))

# crop
model.add(Cropping2D(cropping=((70,25), (0,0))))

In [11]:
# dummy architecture
"""
model.add(Flatten())
model.add(Dense(1))
"""

'\nmodel.add(Flatten())\nmodel.add(Dense(1))\n'

In [ ]:
# nvidia architecture

model.add(Convolution2D(nb_filter=24,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(nb_filter=36,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(nb_filter=48,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))

model.add(Convolution2D(nb_filter=64,  nb_row=3, nb_col=3, activation='relu'))
model.add(Convolution2D(nb_filter=64,  nb_row=3, nb_col=3, activation='relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))

In [ ]:
model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=EPOCHS, verbose=1)

Train on 38572 samples, validate on 9644 samples
Epoch 1/10


In [ ]:
model.save('models/momo2.h5')